In [1]:
import sys
sys.path.append('..')

In [2]:
import os
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP/pythonbooks'

In [3]:
os.chdir('..')
os.getcwd()

'/mnt/3B1D7BDC2D2641B0/Documents/GIT/SQUAD_NLP'

# Хочется вытащить все в питонбук, потому что из консоли работать неудобно

# загрузка данных

In [4]:
# # uncomment to install
# # add whatelse u want

!pip install ujson
!pip install tensorboardX
!pip install spacy
# 

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!python3 setup.py

glove.840B.300d.zip: 2.18GB [2:16:48, 265kB/s]                                  
Unzipping GloVe word vectors...
You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/site-packages/en_core_web_sm
-->
/home/celidos/anaconda3/envs/venv_tensorflowgpu/lib/python3.6/site-packages/spacy/data/en
You can now load the model via spacy.load('en')
Pre-processing train examples...
100%|█████████████████████████████████████████| 442/442 [00:55<00:00,  8.75it/s]
130319 questions in total
Pre-processing word vectors...
2196018it [02:46, 13156.90it/s]                                                 
88744 / 107454 tokens have corresponding word embedding vector
Pre-processing char vectors...
1374 tokens have corresponding char embedd

# Обучение

In [4]:
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as sched
import torch.utils.data as data
import util

from args import get_train_args
from collections import OrderedDict
from json import dumps
from models import BiDAF
from tensorboardX import SummaryWriter
from tqdm import tqdm
from ujson import load as json_load
from util import collate_fn, SQuAD

In [5]:
train_args = type('', (), {})() # empty "structure" where you can add fields

train_args.train_record_file = './data/train.npz'
train_args.dev_record_file = './data/dev.npz'
train_args.test_record_file = './data/test.npz'
train_args.word_emb_file = './data/word_emb.json'
train_args.char_emb_file = './data/char_emb.json'
train_args.train_eval_file = './data/train_eval.json'
train_args.dev_eval_file = './data/dev_eval.json'
train_args.test_eval_file = './data/test_eval.json'

train_args.name = 'train'       # ???
train_args.max_ans_len = 15     # Maximum length of a predicted answer.
train_args.num_workers = 4      # Number of sub-processes to use per data loader.
train_args.save_dir = './save/' # Base directory for saving information.
train_args.batch_size = 64      # Batch size per GPU. Scales automatically when \
                                #    multiple GPUs are available.
train_args.use_squad_v2 = True  # Whether to use SQuAD 2.0 (unanswerable) questions.
train_args.hidden_size = 100    # Number of features in encoder hidden layers
train_args.num_visuals = 10     # Number of examples to visualize in TensorBoard.
train_args.load_path = None     # Path to load as a model checkpoint.

train_args.eval_steps = 50000   # Number of steps between successive evaluations
train_args.lr = 0.5             # learning rate
train_args.l2_wd = 0.0          # L2 weight decay.
train_args.num_epochs = 30      # Number of epochs for which to train. Negative means forever.
train_args.drop_prob = 0.2      # Probability of zeroing an activation in dropout layers.
train_args.metric_name = 'F1'   # choices=('NLL', 'EM', 'F1'), name of dev metric to determine best checkpoint.
train_args.max_checkpoints = 5  # Maximum number of checkpoints to keep on disk.
train_args.max_grad_norm = 5.0  # Maximum gradient norm for gradient clipping.
train_args.seed = 224           # Random seed
train_args.ema_decay = 0.999    # Decay rate for exponential moving average of parameters.

if train_args.metric_name == 'NLL':
    # Best checkpoint is the one that minimizes negative log-likelihood
    train_args.maximize_metric = False
elif train_args.metric_name in ('EM', 'F1'):
    # Best checkpoint is the one that maximizes EM or F1
    train_args.maximize_metric = True

In [6]:
# from train.py

def main(args):
    # Set up logging and devices
    args.save_dir = util.get_save_dir(args.save_dir, args.name, training=True)
    log = util.get_logger(args.save_dir, args.name)
    tbx = SummaryWriter(args.save_dir)
    device, args.gpu_ids = util.get_available_devices()
    log.info('Args: {}'.format(dumps(vars(args), indent=4, sort_keys=True)))
    args.batch_size *= max(1, len(args.gpu_ids))

    # Set random seed
    log.info('Using random seed {}...'.format(args.seed))
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)

    # Get embeddings
    log.info('Loading embeddings...')
    word_vectors = util.torch_from_json(args.word_emb_file)

    # Get model
    log.info('Building model...')
    model = BiDAF(word_vectors=word_vectors,
                  hidden_size=args.hidden_size,
                  drop_prob=args.drop_prob)
    model = nn.DataParallel(model, args.gpu_ids)
    if args.load_path:
        log.info('Loading checkpoint from {}...'.format(args.load_path))
        model, step = util.load_model(model, args.load_path, args.gpu_ids)
    else:
        step = 0
    model = model.to(device)
    model.train()
    ema = util.EMA(model, args.ema_decay)

    # Get saver
    saver = util.CheckpointSaver(args.save_dir,
                                 max_checkpoints=args.max_checkpoints,
                                 metric_name=args.metric_name,
                                 maximize_metric=args.maximize_metric,
                                 log=log)

    # Get optimizer and scheduler
    optimizer = optim.Adadelta(model.parameters(), args.lr,
                               weight_decay=args.l2_wd)
    scheduler = sched.LambdaLR(optimizer, lambda s: 1.)  # Constant LR

    # Get data loader
    log.info('Building dataset...')
    train_dataset = SQuAD(args.train_record_file, args.use_squad_v2)
    train_loader = data.DataLoader(train_dataset,
                                   batch_size=args.batch_size,
                                   shuffle=True,
                                   num_workers=args.num_workers,
                                   collate_fn=collate_fn)
    dev_dataset = SQuAD(args.dev_record_file, args.use_squad_v2)
    dev_loader = data.DataLoader(dev_dataset,
                                 batch_size=args.batch_size,
                                 shuffle=False,
                                 num_workers=args.num_workers,
                                 collate_fn=collate_fn)

    # Train
    log.info('Training...')
    steps_till_eval = args.eval_steps
    epoch = step // len(train_dataset)
    while epoch != args.num_epochs:
        epoch += 1
        log.info('Starting epoch {}...'.format(epoch))
        with torch.enable_grad(), \
                tqdm(total=len(train_loader.dataset)) as progress_bar:
            for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:
                # Setup for forward
                cw_idxs = cw_idxs.to(device)
                qw_idxs = qw_idxs.to(device)
                batch_size = cw_idxs.size(0)
                optimizer.zero_grad()

                # Forward
                log_p1, log_p2 = model(cw_idxs, qw_idxs)
                y1, y2 = y1.to(device), y2.to(device)
                loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
                loss_val = loss.item()

                # Backward
                loss.backward()
                nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step(step // batch_size)
                ema(model, step // batch_size)

                # Log info
                step += batch_size
                progress_bar.update(batch_size)
                progress_bar.set_postfix(epoch=epoch,
                                         NLL=loss_val)
                tbx.add_scalar('train/NLL', loss_val, step)
                tbx.add_scalar('train/LR',
                               optimizer.param_groups[0]['lr'],
                               step)

                steps_till_eval -= batch_size
                if steps_till_eval <= 0:
                    steps_till_eval = args.eval_steps

                    # Evaluate and save checkpoint
                    log.info('Evaluating at step {}...'.format(step))
                    ema.assign(model)
                    results, pred_dict = evaluate(model, dev_loader, device,
                                                  args.dev_eval_file,
                                                  args.max_ans_len,
                                                  args.use_squad_v2)
                    saver.save(step, model, results[args.metric_name], device)
                    ema.resume(model)

                    # Log to console
                    results_str = ', '.join('{}: {:05.2f}'.format(k, v)
                                            for k, v in results.items())
                    log.info('Dev {}'.format(results_str))

                    # Log to TensorBoard
                    log.info('Visualizing in TensorBoard...')
                    for k, v in results.items():
                        tbx.add_scalar('dev/{}'.format(k), v, step)
                    util.visualize(tbx,
                                   pred_dict=pred_dict,
                                   eval_path=args.dev_eval_file,
                                   step=step,
                                   split='dev',
                                   num_visuals=args.num_visuals)

In [7]:
def evaluate(model, data_loader, device, eval_file, max_len, use_squad_v2):
    nll_meter = util.AverageMeter()

    model.eval()
    pred_dict = {}
    with open(eval_file, 'r') as fh:
        gold_dict = json_load(fh)
    with torch.no_grad(), \
            tqdm(total=len(data_loader.dataset)) as progress_bar:
        for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in data_loader:
            # Setup for forward
            cw_idxs = cw_idxs.to(device)
            qw_idxs = qw_idxs.to(device)
            batch_size = cw_idxs.size(0)

            # Forward
            log_p1, log_p2 = model(cw_idxs, qw_idxs)
            y1, y2 = y1.to(device), y2.to(device)
            loss = F.nll_loss(log_p1, y1) + F.nll_loss(log_p2, y2)
            nll_meter.update(loss.item(), batch_size)

            # Get F1 and EM scores
            p1, p2 = log_p1.exp(), log_p2.exp()
            starts, ends = util.discretize(p1, p2, max_len, use_squad_v2)

            # Log info
            progress_bar.update(batch_size)
            progress_bar.set_postfix(NLL=nll_meter.avg)

            preds, _ = util.convert_tokens(gold_dict,
                                           ids.tolist(),
                                           starts.tolist(),
                                           ends.tolist(),
                                           use_squad_v2)
            pred_dict.update(preds)

    model.train()

    results = util.eval_dicts(gold_dict, pred_dict, use_squad_v2)
    results_list = [('NLL', nll_meter.avg),
                    ('F1', results['F1']),
                    ('EM', results['EM'])]
    if use_squad_v2:
        results_list.append(('AvNA', results['AvNA']))
    results = OrderedDict(results_list)

    return results, pred_dict

In [10]:
main(train_args)

[04.24.19 01:59:03] Args: {
    "batch_size": 64,
    "char_emb_file": "./data/char_emb.json",
    "dev_eval_file": "./data/dev_eval.json",
    "dev_record_file": "./data/dev.npz",
    "drop_prob": 0.2,
    "ema_decay": 0.999,
    "eval_steps": 50000,
    "gpu_ids": [
        0
    ],
    "hidden_size": 100,
    "l2_wd": 0.0,
    "load_path": null,
    "lr": 0.5,
    "max_ans_len": 15,
    "max_checkpoints": 5,
    "max_grad_norm": 5.0,
    "maximize_metric": true,
    "metric_name": "F1",
    "name": "train",
    "num_epochs": 30,
    "num_visuals": 10,
    "num_workers": 4,
    "save_dir": "./save/train/train-01",
    "seed": 224,
    "test_eval_file": "./data/test_eval.json",
    "test_record_file": "./data/test.npz",
    "train_eval_file": "./data/train_eval.json",
    "train_record_file": "./data/train.npz",
    "use_squad_v2": true,
    "word_emb_file": "./data/word_emb.json"
}
[04.24.19 01:59:03] Using random seed 224...
[04.24.19 01:59:03] Loading embeddings...
[04.24.19 01:59:

  0%|          | 0/129922 [00:00<?, ?it/s]

[04.24.19 02:00:08] Training...
[04.24.19 02:00:08] Starting epoch 1...


 39%|███▊      | 50048/129922 [05:16<08:57, 148.63it/s, NLL=6.21, epoch=1]

[04.24.19 02:05:25] Evaluating at step 50048...


...

 66%|██████▌   | 86016/129922 [09:11<04:27, 164.00it/s, NLL=2.12, epoch=30] 

[04.24.19 08:56:57] Evaluating at step 3853754...



 66%|██████▌   | 86016/129922 [09:18<04:27, 164.00it/s, NLL=2.12, epoch=30]

[04.24.19 08:57:04] Saved checkpoint: ./save/train/train-01/step_3853754.pth.tar
[04.24.19 08:57:04] Removed checkpoint: ./save/train/train-01/step_3853754.pth.tar
[04.24.19 08:57:04] Dev NLL: 03.18, F1: 60.51, EM: 56.88, AvNA: 67.57
[04.24.19 08:57:04] Visualizing in TensorBoard...


100%|██████████| 129922/129922 [13:56<00:00, 155.23it/s, NLL=1.09, epoch=30]


In [9]:
train_dataset = SQuAD(train_args.train_record_file, train_args.use_squad_v2)
train_loader = data.DataLoader(train_dataset,
                               batch_size=train_args.batch_size,
                               shuffle=True,
                               num_workers=train_args.num_workers,
                               collate_fn=collate_fn)
for cw_idxs, cc_idxs, qw_idxs, qc_idxs, y1, y2, ids in train_loader:
    print(cc_idxs.shape)
    break

torch.Size([64, 353, 16])


In [5]:
util.get_available_devices()

(device(type='cuda', index=0), [0])